In [1]:
import os
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import pymc3
import matplotlib.pyplot as plt
from plotnine import *
import itertools
import geopy
import geopandas
import statsmodels.api as sm
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from fbprophet import Prophet
#geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
url = 'https://opendata.arcgis.com/datasets/3f6982695aca45af9fe48fcd59360219_0.geojson'

In [2]:
r = requests.get(url).json()
temp = []
for i in range(0, len(r['features'])):
    temp.append(r['features'][i]['properties'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/justingoss/Desktop/sac_animals/sac_animals/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-487e95436619>", line 1, in <module>
    r = requests.get(url).json()
  File "/Users/justingoss/Desktop/sac_animals/sac_animals/lib/python3.9/site-packages/requests/models.py", line 891, in json
    return complexjson.loads(
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
KeyboardInterrupt

During handling of the above except

TypeError: object of type 'NoneType' has no len()

In [ ]:
df = pd.json_normalize(temp)
df.columns =  df.columns.str.replace('attributes.', '')
df.columns = [x.lower() for x in df.columns]
df = df.set_index('objectid')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.intake_type.value_counts(normalize=True).to_frame()

In [ ]:
df.animal_sex.value_counts()

In [ ]:
df.loc[df['animal_sex'].isin(['M', 'N']), 'animal_sex_clean'] = 'Male'
df.loc[df['animal_sex'].isin(['S', 'F']), 'animal_sex_clean'] = 'Female'
df.loc[df['animal_sex'].isin(['U']), 'animal_sex_clean'] = 'Unknown'

df.loc[df['animal_sex'].isin(['N', 'S']), 'animal_intake_status'] = 'neutured'
df.loc[df['animal_sex'].isin(['M', 'F']), 'animal_intake_status'] = 'intact'
df.loc[df['animal_sex'].isin(['U']), 'animal_intake_status'] = 'Unknown'

In [ ]:
df['name_unknown'] = np.where(df['animal_name'].isnull(), 1, 0)
df['name_unknown'] = df['name_unknown'].fillna('Unknown')

In [ ]:
df['has_activity_number'] = np.where(df['activity_number'].isnull(), 0, 1)

In [ ]:
df['animal_age'] = df['animal_age'].fillna('Unknown')
df['animal_age_years'] = df.loc[df['animal_age'].str.contains('Y'), 'animal_age']
df['animal_age_months'] = df.loc[df['animal_age'].str.contains('M'), 'animal_age']
df['animal_age_days'] = df.loc[df['animal_age'].str.contains('D'), 'animal_age']

df['animal_age_years'] = df['animal_age_years'].str.extract('(\d+)', expand=False).fillna(0)
df['animal_age_months'] = df['animal_age_months'].str.extract('(\d+)', expand=False).fillna(0)
df['animal_age_days'] = df['animal_age_days'].str.extract('(\d+)', expand=False).fillna(0)

df['animal_age_years'] = pd.to_numeric(df['animal_age_years'])
df['animal_age_months'] = pd.to_numeric(df['animal_age_months'])
df['animal_age_days'] = pd.to_numeric(df['animal_age_days'])

df['animal_age_in_days'] = (df['animal_age_years'] * 365) + (df['animal_age_months'] * 30) + (df['animal_age_days'])

In [ ]:
df = df.drop(['animal_age_years', 'animal_age_months', 'animal_age_days'], axis=1)

In [ ]:
df.outcome_type.value_counts()

In [ ]:
df.loc[df['outcome_type'].isin(['ADOPTION', 'FOSTER', 'RTO', 'RTF']), 'outcome_group'] = 'to_owner'
df.loc[df['outcome_type'].isin(['EUTH', 'DIED', 'EUTH VET', 'DISPOSAL', 'EUTH FIELD']), 'outcome_group'] = 'dead'
df.loc[df['outcome_type'].isin(['TRANSFER', 'MISSING', 'ESCAPED', 'STOLEN', 'AT RISK', 'SUBMIT TO']), 'outcome_group'] = 'other'
df['outcome_group'] = df['outcome_group'].dropna()

In [ ]:
pd.crosstab(df.outcome_group, df.outcome_subtype)

In [ ]:
df.intake_type.value_counts()
df.loc[df['intake_type'].isin(['STRAY', 'OWNER SUR']), 'intake_type_clean'] = 'unowned'
df.loc[df['intake_type'].isin(['FOSTER', 'RETURN', 'QUARANTINE']), 'intake_type_clean'] = 'owned_safe'
df.loc[df['intake_type'].isin(['CONFISCATE', 'HOMELESS', 'PR CUSTODY']), 'intake_type_clean'] = 'owned_unsafe'
df.loc[df['intake_type'].isin(['MEDICAL', 'EUTH REQ', 'TRANSFER', 'DISPO REQ']), 'intake_type_clean'] = 'other'

In [ ]:
df.loc[df['animal_id'] == 'A540762'].sort_values('intake_date')

In [ ]:
df.animal_primarycolor.unique()

In [ ]:
df['animal_primarycolor'] = df['animal_primarycolor'].str.strip()
df.loc[df['animal_primarycolor'].isin(['BLACK', 'BLK SMOKE']), 'color_clean'] = 'black'
df.loc[df['animal_primarycolor'].isin(['RED', 'FLAME PT', 'ORANGE', 'APRICOT']), 'color_clean'] = 'orange'
df.loc[df['animal_primarycolor'].isin(['BROWN', 'TAN', 'GRAY', 'WHITE', 'YELLOW',
                                      'FAWN', 'BLUE', 'CHOCOLATE', 'CREAM',
                                      'BLONDE', 'GOLD', 'BUFF', 'SILVER',
                                      'BEIGE', 'BLUE SMOKE']), 'color_clean'] = 'other_solid'
df.loc[df['animal_primarycolor'].isin(['BRN TABBY', 'GRAY TABBY', 'BLK TABBY',
                                      'ORG TABBY', 'CRM TABBY', 'BLUE TABBY', 'SLVR TABBY']), 'color_clean'] = 'tabby'
df.loc[df['animal_primarycolor'].isin(['LYNX PT', 'SEAL PT', 'TORTIE (DILUTE)', 'CALICO', 'TORTIE', 'BLUE MERLE',
                                      'SL LYNX PT', 'BLUE PT', 'CALICO PT', 'TORTIE PT', 'CHOC PT', 'LILAC PT',
                                      'TRICOLOR', 'CALICO (DILUTE)', 'ORG TIGER', 'HARLEQUIN', 'BRN TIGER',
                                      'BLK TIGER', 'CRM TIGER', 'BRN BRINDLE',
                                      'BL BRINDLE', 'BLUE BRINDLE', 'GRAY TIGER']), 'color_clean'] = 'other_pattern'
df['color_clean'] = df['color_clean'].fillna('other')

In [ ]:
df.color_clean.value_counts()

In [ ]:
df['intake_times'] = df.groupby('animal_id').animal_id.transform('count')

In [ ]:
df.sort_values('intake_times')

In [ ]:
df['intake_date'] = pd.to_datetime(df['intake_date'])
df['outcome_date'] = pd.to_datetime(df['outcome_date'])
df['n'] = 1

In [ ]:
df_dt = df.set_index('intake_date')
y = df_dt['n'].resample('W').sum()
y = y[(y.index > '2017-12-03') & (y.index < '2020-12-06')]

In [ ]:
y.plot()
plt.show()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(y, model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 52) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
mod = sm.tsa.statespace.SARIMAX(y,
                                order=(0, 1, 1),
                                seasonal_order=(0, 1, 1, 52),
                                enforce_stationarity=True,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(16, 8))
plt.show()

In [ ]:
pred = results.get_prediction(start=pd.to_datetime('2020-07-05 00:00:00+00:00'), dynamic=False)
pred_ci = pred.conf_int()
ax = y['2017':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
pred_uc = results.get_forecast(steps=100)
pred_ci = pred_uc.conf_int()
ax = y.plot(label='observed', figsize=(14, 7))
pred_uc.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()

In [ ]:
p_model = Prophet(interval_width=0.95, weekly_seasonality=True)
y_prophet = y.reset_index().rename(columns={'intake_date': 'ds', 'n': 'y'})
y_prophet['ds'] = y_prophet['ds'].dt.tz_localize(None)
p_model.fit(y_prophet)
y_forecast = p_model.make_future_dataframe(periods=36, freq='W')
y_forecast = p_model.predict(y_forecast)

In [ ]:
plt.figure(figsize=(18, 6))
p_model.plot(y_forecast, xlabel = 'Date', ylabel = 'Sales')

In [ ]:
df['days_spent'] = (df['outcome_date'] - df['intake_date']).dt.days

In [ ]:
df.head()

In [ ]:
#df.to_csv('sac_animals.csv')